<a href="https://colab.research.google.com/github/skywalker0803r/Job-OEM/blob/main/Store_Sales_Time_Series_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from statsmodels.graphics import tsaplots
from sklearn.multioutput import RegressorChain, MultiOutputRegressor

In [53]:
holidays_events = pd.read_csv(
    '/content/drive/MyDrive/store-sales-time-series-forecasting/holidays_events.csv',
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
holidays_events = holidays_events.set_index('date').to_period('D')

store_sales = pd.read_csv(
   '/content/drive/MyDrive/store-sales-time-series-forecasting/train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()
sales = store_sales.copy()
store_sales

sales  onpromotion
store_nbr family     date                              
1         AUTOMOTIVE 2013-01-01   0.000000            0
                     2013-01-02   2.000000            0
                     2013-01-03   3.000000            0
                     2013-01-04   3.000000            0
                     2013-01-05   5.000000            0
...                                    ...          ...
9         SEAFOOD    2017-08-11  23.830999            0
                     2017-08-12  16.859001            4
                     2017-08-13  20.000000            0
                     2017-08-14  17.000000            0
                     2017-08-15  16.000000            0

[3000888 rows x 2 columns]

In [54]:
sales.reset_index(inplace=True)
sales['dayofweek'] = sales['date'].dt.dayofweek
# sales

In [55]:
seasonal = sales.groupby(['family','dayofweek']).sum().reset_index()
# seasonal

In [58]:
dayOfWeekfig = go.Figure()
for fam in seasonal['family'].unique():
    x = seasonal[seasonal['family'] == fam]['dayofweek'].values.tolist()
    y = seasonal[seasonal['family'] == fam]['sales'].values.tolist()
    dayOfWeekfig.add_trace(go.Scatter(x=x, y=y, name= fam))
    
dayOfWeekfig.update_layout(title='Sales per family', xaxis_title='Day of Week', yaxis_title='Sales(M)')
dayOfWeekfig.show()

In [59]:
average_sales = (sales.groupby('date').mean().squeeze())
# average_sales

In [60]:
x = average_sales.reset_index()['date'].astype('str').values.tolist()
y = average_sales['sales'].values.tolist()
trendfig = go.Figure()
trendfig.add_trace(go.Scatter(x=x, y=y, name='Average Sales'))
trendfig.update_layout(title='Sales', xaxis_title='Date', yaxis_title='Sales(M)')
trendfig.show()

In [61]:
moving_avg_365 = average_sales['sales'].rolling(window=365, center=False).mean().reset_index()
# moving_avg_365

In [62]:
x = moving_avg_365['date'].astype('str').values.tolist()
y = moving_avg_365['sales'].values.tolist()
trendfig.add_trace(go.Scatter(x=x, y=y, name='Moving Average 365'))
trendfig.show()

In [63]:
dp = DeterministicProcess(
    index=average_sales.index,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=1,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
)
# `in_sample` creates features for the dates given in the `index` argument
X = dp.in_sample()
y = average_sales['sales']
# X.head()

In [64]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X,y)

y_pred = pd.Series(lr.predict(X), index=X.index)

In [65]:
X = dp.out_of_sample(steps=180)
y_fore = pd.Series(lr.predict(X), index=X.index).reset_index().set_axis(['date', 'sales'], axis=1)
# y_fore

In [66]:
x = y_fore['date'].astype('str').values.tolist()
y = y_fore['sales'].values.tolist()
trendfig.add_trace(go.Scatter(x=x, y=y, name='Prediction'))
trendfig.show()

In [67]:
fourier = CalendarFourier(freq="A", order=10)  # 10 sin/cos pairs for "A"nnual seasonality

dp = DeterministicProcess(
    index=average_sales.index,
    constant=True,   # dummy feature for bias (y-intercept)
    order=1,         # trend ( order 1 means linear)
    seasonal=True,   # weekly seasonality (indicators)
    additional_terms=[fourier], # annual seasonality
    drop=True,       # drop terms to avoid collinearity
)

X = dp.in_sample() # create features for dates in tunnel.index
y = average_sales["sales"]
#X.head()

In [68]:
lr = LinearRegression().fit(X,y)
y_pred = pd.Series(lr.predict(X), index=X.index)
y_deseason = y - y_pred

X_fore = dp.out_of_sample(steps=180)
y_fore = pd.Series(lr.predict(X_fore), index=X_fore.index).reset_index().set_axis(['date', 'sales'], axis=1)

seasonalfig = go.Figure()
seasonalfig.add_trace(go.Scatter(x=y.reset_index()['date'].astype('str').values.tolist(), 
                                 y=y.values.tolist(),
                                 mode='lines+markers',
                                 name='Average Sales'))
seasonalfig.add_trace(go.Scatter(x=y_pred.reset_index()['date'].astype('str').values.tolist(),
                                 y=y_pred.values.tolist(),
                                 name='seasonal'))
seasonalfig.add_trace(go.Scatter(x=y_deseason.reset_index()['date'].astype('str').values.tolist(),
                                 y=y_deseason.values.tolist(),
                                 name='deseasonal'))
seasonalfig.add_trace(go.Scatter(x=y_fore['date'].astype('str').values.tolist(),
                                 y=y_fore['sales'].values.tolist(),
                                 name='predict'))
seasonalfig.update_layout(title='Seasonal')
seasonalfig.show()

In [69]:
family_sales = (
    store_sales
    .groupby(['store_nbr','family', 'date'])
    .mean() 
    .unstack(['family', 'store_nbr'])
    .loc['2017', ['sales','onpromotion']]
)
# family_sales = (
#     store_sales
#     .groupby(['family', 'date'])
#     .mean() 
#     .unstack('family')
#     .loc['2017', ['sales','onpromotion']]
# )

# display(family_sales)
# display(store_sales)
df_sales = family_sales.loc(axis=1)[:, 'MAGAZINES']
display(df_sales)

sales                                               ...  \
family     MAGAZINES                                               ...   
store_nbr          1   10   11   12   13   14   15   16   17   18  ...   
date                                                               ...   
2017-01-01       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
2017-01-02       1.0  1.0  5.0  3.0  1.0  4.0  1.0  1.0  7.0  2.0  ...   
2017-01-03       1.0  0.0  3.0  1.0  0.0  1.0  1.0  0.0  2.0  2.0  ...   
2017-01-04       6.0  0.0  2.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  ...   
2017-01-05       6.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  4.0  1.0  ...   
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2017-08-11       4.0  1.0  1.0  2.0  0.0  2.0  2.0  4.0  8.0  6.0  ...   
2017-08-12       6.0  0.0  2.0  0.0  0.0  4.0  2.0  2.0  8.0  2.0  ...   
2017-08-13       4.0  1.0  1.0  1.0  1.0  3.0  2.0  0.0  4.0  2.0  ...   
2017-08-14       8.0  0.0  3.0  3.0  1.0  4.0  3.0  2.0  6.0  5.0  ...   
2017-08-15       3.0  1.0  4.0  4.0  1.0  5.0  1.0  1.0  5.0  2.0  ...   

           onpromotion                                               
family       MAGAZINES                                               
store_nbr            5   50   51   52   53   54    6    7    8    9  
date                                                                 
2017-01-01         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2017-01-02         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2017-01-03         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2017-01-04         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2017-01-05         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...                ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
2017-08-11         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2017-08-12         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2017-08-13         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2017-08-14         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2017-08-15         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[227 rows x 108 columns]

In [70]:
# Target series
y = family_sales.loc[:, 'sales']
# display(y)

# X_1: Features for Linear Regression
fourier = CalendarFourier(freq='D', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=5,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X_1 = dp.in_sample()


# X_2: Features for XGBoost
X_2 = family_sales.drop('sales', axis=1).stack()  # onpromotion feature
# display(X_2)
# Label encoding for 'family'
le = LabelEncoder()  # from sklearn.preprocessing
X_2 = X_2.stack()
X_2 = X_2.reset_index(['family', 'store_nbr'])

X_2['family'] = le.fit_transform(X_2['family'])

# Label encoding for seasonality
X_2["day"] = X_2.index.day  # values are day of the month

display(X_1)
display(X_2)

,const,trend,trend_squared,trend_cubed,trend**4,trend**5,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)"
date,,,,,,,,,,,,
2017-01-01,1.0,1.0,1.0,1.0,1.000000e+00,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-02,1.0,2.0,4.0,8.0,1.600000e+01,3.200000e+01,1.0,0.0,0.0,0.0,0.0,0.0
2017-01-03,1.0,3.0,9.0,27.0,8.100000e+01,2.430000e+02,0.0,1.0,0.0,0.0,0.0,0.0
2017-01-04,1.0,4.0,16.0,64.0,2.560000e+02,1.024000e+03,0.0,0.0,1.0,0.0,0.0,0.0
2017-01-05,1.0,5.0,25.0,125.0,6.250000e+02,3.125000e+03,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,1.0,223.0,49729.0,11089567.0,2.472973e+09,5.514731e+11,0.0,0.0,0.0,0.0,1.0,0.0
2017-08-12,1.0,224.0,50176.0,11239424.0,2.517631e+09,5.639493e+11,0.0,0.0,0.0,0.0,0.0,1.0
2017-08-13,1.0,225.0,50625.0,11390625.0,2.562891e+09,5.766504e+11,0.0,0.0,0.0,0.0,0.0,0.0


,store_nbr,family,onpromotion,day
date,,,,
2017-01-01,1,0,0.0,1
2017-01-01,1,1,0.0,1
2017-01-01,1,2,0.0,1
2017-01-01,1,3,0.0,1
2017-01-01,1,4,0.0,1
...,...,...,...,...
2017-08-15,9,28,0.0,15
2017-08-15,9,29,1.0,15
2017-08-15,9,30,148.0,15


In [71]:
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None  # store column names from fit method
    
    def fit(self, X_1, X_2, y):
        # Train model_1
        self.model_1.fit(X_1, y)

        # Make predictions
        y_fit = pd.DataFrame(
            self.model_1.predict(X_1), 
            index=X_1.index, 
            columns=y.columns,
        )

        # Compute residuals
        y_resid = y - y_fit
        y_resid = y_resid.stack().squeeze() # wide to long
#         display(y_resid.stack())
        # Train model_2 on residuals
        self.model_2.fit(X_2, y_resid.stack())

        # Save column names for predict method
        self.y_columns = y.columns
        # Save data for question checking
        self.y_fit = y_fit
        self.y_resid = y_resid
        
    def predict(self, X_1, X_2):
        # Predict with model_1
        y_pred = pd.DataFrame(
            self.model_1.predict(X_1), 
            index=X_1.index, columns=self.y_columns,
        )
#         display('model 1 predict', self.model_1.predict(X_1), self.model_1.predict(X_1).shape)
#         display('y_pred before stack squeeze', y_pred)
        y_pred = y_pred.stack().squeeze()  # wide to long

        # Add model_2 predictions to model_1 predictions
#         display('y_pred after stack squeeze', y_pred)
#         display('model 2 predict', np.reshape(self.model_2.predict(X_2), (-1,33)))
        y_pred += np.reshape(self.model_2.predict(X_2), (-1, 33))
#         display(y_pred.unstack())
        return y_pred.unstack()

In [77]:
model = BoostedHybrid(
    model_1=Ridge(),
    model_2=KNeighborsRegressor(),
)

model.fit(X_1, X_2, y)
y_pred = model.predict(X_1, X_2)

y_pred = y_pred.clip(0.0)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning:

Ill-conditioned matrix (rcond=1.86813e-25): result may not be accurate.

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:964: FutureWarning:

Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:566: FutureWarning:

Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.



In [78]:
y_train, y_valid = y[:"2017-07-01"], y["2017-07-02":]
X1_train, X1_valid = X_1[: "2017-07-01"], X_1["2017-07-02" :]
X2_train, X2_valid = X_2.loc[:"2017-07-01"], X_2.loc["2017-07-02":]

# model.fit(X1_train, X2_train, y_train)
y_fit = model.predict(X1_train, X2_train).clip(0.0)
y_pred = model.predict(X1_valid, X2_valid).clip(0.0)

families = y.columns[0:6]

# display(y.loc(axis=1)[families])
# display(y_fit.loc(axis=1)[families])
# display(y_pred.loc(axis=1)[families])

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:566: FutureWarning:

Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:566: FutureWarning:

Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.



In [79]:
total_training = 0
total_validation = 0
for fam in families:
    fam_string = " ".join(fam)
    famplot = go.Figure()
    x_train_val = y.loc(axis=1)[families].index.astype('str')
    y_train_val = y.loc(axis=1)[families][fam].values
#     print(f'x_train_val : {x_train_val} \n y_train_val : {y_train_val} \n fam : {fam_string}')
    famplot.add_trace(go.Scatter(x=x_train_val,y=y_train_val, name=fam_string+"-sales", mode='lines+markers'))
    
    x_fit_val = y_fit.loc(axis=1)[families].index.astype('str')
    y_fit_val = y_fit.loc(axis=1)[families][fam].values
    famplot.add_trace(go.Scatter(x=x_fit_val,y=y_fit_val, name=fam_string+"-fit"))
    
    x_predict = y_pred.loc(axis=1)[families].index.astype('str')
    y_predict = y_pred.loc(axis=1)[families][fam].values
    famplot.add_trace(go.Scatter(x=x_predict,y=y_predict, name=fam_string+"-predict"))
    famplot.update_layout(title=fam_string)
    famplot.show()

    rmsle_train = mean_squared_log_error(y_train_val[:len(y_fit_val)], y_fit_val) ** 0.5
    rmsle_valid = mean_squared_log_error(y_train_val[:len(y_predict)], y_predict) ** 0.5
    print(f'Training RMSLE for {fam} : {rmsle_train:.5f}')
    print(f'Validation RMSLE for {fam} : {rmsle_valid:.5f}')
    total_training += rmsle_train
    total_validation += rmsle_valid
    
print(f'Total _training : {total_training}')
print(f'Total _validation : {total_validation}')

Training RMSLE for ('AUTOMOTIVE', '1') : 0.50442
Validation RMSLE for ('AUTOMOTIVE', '1') : 0.66296


Training RMSLE for ('BABY CARE', '1') : 0.03952
Validation RMSLE for ('BABY CARE', '1') : 0.04588


Training RMSLE for ('BEAUTY', '1') : 0.44444
Validation RMSLE for ('BEAUTY', '1') : 0.75957


Training RMSLE for ('BEVERAGES', '1') : 0.55740
Validation RMSLE for ('BEVERAGES', '1') : 1.13580


Training RMSLE for ('BOOKS', '1') : 0.39395
Validation RMSLE for ('BOOKS', '1') : 0.57192


Training RMSLE for ('BREAD/BAKERY', '1') : 0.44099
Validation RMSLE for ('BREAD/BAKERY', '1') : 0.83738
Total _training : 2.3807179472551545
Total _validation : 4.013510332192631


In [81]:
df_test = pd.read_csv(
    '/content/drive/MyDrive/store-sales-time-series-forecasting/test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()
# df_test

In [82]:
y = df_test.unstack(['store_nbr', 'family']).loc["2017"]
# print(y)
# Create training data
# fourier = CalendarFourier(freq='M', order=4)
# dp = DeterministicProcess(
#     index=y.index,
#     constant=True,
#     order=5,
#     seasonal=True,
#     additional_terms=[fourier],
#     drop=True,
# )
# X = dp.in_sample()

fourier = CalendarFourier(freq='D', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=5,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)

X_1 = dp.in_sample()
# display(X_1)
# X['NewYear'] = (X.index.dayofyear == 1)

In [83]:
# X_1: Features for Linear Regression
X_1 = dp.in_sample()

# X_2: Features for XGBoost
X_2 = df_test  # onpromotion feature

# Label encoding for 'family'
le = LabelEncoder()  # from sklearn.preprocessing
X_2 = X_2.reset_index('family')

X_2['family'] = le.fit_transform(X_2['family'])
X_2.reset_index(inplace=True)
# display(X_2['date'].dt.day)
# Label encoding for seasonality
X_2["day"] = X_2['date'].dt.day  # values are day of the month
X_2 = X_2.groupby(['date', 'family', 'store_nbr']).sum()
X_2 = X_2.reset_index().set_index('date')[['family', 'onpromotion', 'day', 'store_nbr']]
# display(X_1)
# display(X_2)

model.predict(X_1, X_2)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:566: FutureWarning:

Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.



family      AUTOMOTIVE                                                  \
store_nbr            1          10          11          12          13   
date                                                                     
2017-08-16   -5.318682   -1.911125   46.522914  113.823792  601.325995   
2017-08-17 -103.833581   81.125735   38.856512    9.034239  600.969833   
2017-08-18  155.709944  -21.323037   25.750277   40.519451  599.713994   
2017-08-19  155.964760   -4.776664   12.091729   41.371861 -143.176058   
2017-08-20  408.698698   -4.681860  -29.097784  -67.201253 -110.556284   
2017-08-21  414.161039   54.237079   31.874338   33.657803  -31.756158   
2017-08-22   45.031146  -44.432423   52.862356   37.925770  119.658136   
2017-08-23  266.817490   17.342142  -33.304231  -17.508191 -148.333054   
2017-08-24  327.915080  167.417398   69.495762  -51.886210  -81.302992   
2017-08-25  464.528704  132.918912  143.413431  -53.153873  -17.735602   
2017-08-26  413.089977  249.699083  145.819995  -52.297331  -38.206109   
2017-08-27  329.615133  115.470990  -35.476336  -48.272182 -264.534448   
2017-08-28  246.883824  297.369969 -146.723972  -48.668614 -118.537515   
2017-08-29  299.095292  350.360904  237.118164   49.281621 -115.596003   
2017-08-30    2.471961 -144.817446  475.456958   47.629742 -105.686150   
2017-08-31    4.154922 -158.607852  496.178053 -103.734041 -286.784763   

family                                                                 ...  \
store_nbr           14         15          16          17          18  ...   
date                                                                   ...   
2017-08-16   22.508722  -8.898647 -498.499127   17.477179   -7.140043  ...   
2017-08-17   19.477534   6.293606 -704.547891    1.850865   -9.996678  ...   
2017-08-18   31.993207  -8.097501 -736.351867   41.261670   -9.951537  ...   
2017-08-19   49.623691  23.870562 -321.766815  125.629323   19.297976  ...   
2017-08-20    6.445223 -10.625129   20.495830  136.636672   -5.381136  ...   
2017-08-21    2.388805  27.289595   80.927463   46.712007   10.629770  ...   
2017-08-22   41.349390  22.545495  179.113776  156.918470  -56.999135  ...   
2017-08-23 -445.832387  32.872317  145.087260   94.154766  -81.349653  ...   
2017-08-24 -553.517041  -1.606773  -46.440990   90.407989   -8.880424  ...   
2017-08-25 -215.978071  -2.079380 -210.533413  -43.064678   -8.775932  ...   
2017-08-26 -106.076882  -2.463853   74.394296   33.975160  -15.288936  ...   
2017-08-27  -13.587882  -2.748600 -533.056732   33.293654  -70.426819  ...   
2017-08-28  108.173167  21.495724 -205.274628 -100.757306  -94.115982  ...   
2017-08-29 -229.886603  17.366951 -163.589886  -70.899112 -202.948884  ...   
2017-08-30  -12.596607  17.075039 -498.609974 -234.492499 -203.434053  ...   
2017-08-31  -77.661364  16.926481 -299.498150 -305.045748 -413.064147  ...   

family         SEAFOOD                                                  \
store_nbr            5          50          51          52          53   
date                                                                     
2017-08-16  125.726895 -464.263785  510.845508 -163.291305 -160.744816   
2017-08-17  181.971847 -246.065265  505.344207 -280.731252   17.230195   
2017-08-18  246.940510  -80.667331  293.984924 -525.263135  284.007389   
2017-08-19  246.572996   44.536227  486.632949 -275.300003  125.655904   
2017-08-20  504.766080 -167.297679  472.949020 -480.322226  107.632897   
2017-08-21  236.674045 -207.501648  512.748461 -396.300997 -130.458369   
2017-08-22  113.037819 -242.326753  430.352474 -304.452758  -25.516012   
2017-08-23  -66.635833  -93.991542  433.524124 -415.344018 -120.962026   
2017-08-24 -309.815838 -248.392061  207.512719 -224.269701 -126.561529   
2017-08-25 -193.162025  -93.036605   58.006972 -356.240452 -115.795177   
2017-08-26  293.630673 -172.413434  139.236453 -373.779269  492.779630   
2017-08-27  240.257183 -101.507286  125.717748 -568.427303  399.965034   


In [84]:
y_submit = pd.DataFrame(model.predict(X_1, X_2), index=X_1.index)
y_submit = y_submit.stack(['store_nbr', 'family'])

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:566: FutureWarning:

Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.



In [85]:
y_submit.reset_index()
df_test.reset_index()
results = pd.merge(y_submit.reset_index(), df_test.reset_index(), on=['store_nbr', 'family', 'date'])[['id', 0]]
results = results.set_axis(['id', 'sales'], axis=1)
results.to_csv('submission.csv', index=False)